# Example Use for Quartz Solar Site Forecasts

* Site Documentation: https://api-site.quartz.solar/swagger
* Additional Site Documentation: https://api-site.quartz.solar/docs
* API Access: https://openclimatefix.notion.site/API-Access-2d8d2f64215d4432be830cbcc9220012

## Getting Your Access Token

In [ ]:
import requests
import json
import pprint

client_id = "QLMdXCCHMS9Zl5W2tDG3otpdcY7GeEnJ"
username="username" # Change to be Your username
pwd = "pwd" # Change to be Your password
domain = "nowcasting-pro.eu.auth0.com"
grant_type="password"

url = f"https://{domain}/oauth/token"
header = {'content-type': 'application/json'}
data = json.dumps({"client_id":client_id,"username":username,"password":pwd,"grant_type":grant_type,"audience":"https://api.nowcasting.io/"})

r = requests.post(url, data=data, headers=header)
try:
    access_token = r.json()['access_token']
except KeyError:
    access_token = None
    print("Failed to retrieve access token.")

print("Response status code:", r.status_code)
print("Response headers:", r.headers)
print("Response content:", r.content)

### Or load access token from .env file

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
access_token = os.getenv('ACCESS_TOKEN')

In [ ]:
# Set the URL to the API
base_url = "https://api-site.quartz.solar"

## Get Sites currently in the system under the user

In [ ]:
# If you have not sites the response will be an empty list
r = requests.get(url=(base_url + '/sites'), headers={"Authorization": "Bearer " + access_token})

data = r.json()
pprint.pprint(data)

In [ ]:
# Print all of the site uuids in the system
site_uuids = [site['site_uuid'] for site in data['site_list']]
print(site_uuids)

## Add a new site to the system

In [ ]:
headers = {
    "Authorization": "Bearer " + access_token,
    "Content-Type": "application/json"
}

new_site_data = {
  "client_site_id": 1,
  "client_site_name": "test_site_1",
  "orientation": 15,
  "tilt": 35,
  "latitude": 53.68,
  "longitude": -2.00,
  "inverter_capacity_kw": 4,
  "module_capacity_kw": 4
}

response = requests.post(base_url + "/sites", headers=headers, json=new_site_data)
if response.status_code == 201:
    print("Site added successfully:")
    pprint.pprint(response.json())
else:
    print("Failed to add site. Status code:", response.status_code, "Response:", response.json())

## Get the forecast from the a site

In [ ]:
# site_uuid = site_uuids[1]
site_uuid = 'INSERT_SITE_UUID'

r = requests.get(url=(base_url + f'/sites/{site_uuid}/pv_forecast'), headers={"Authorization": "Bearer " + access_token})
data = r.json()
pprint.pprint(data)

In [ ]:
import plotly.graph_objects as go

# Extracting the relevant data for plotting
target_datetime_utc = [forecast['target_datetime_utc'] for forecast in data['forecast_values']]
expected_generation_kw = [forecast['expected_generation_kw'] for forecast in data['forecast_values']]

# Plotting the data using Plotly
fig = go.Figure(data=go.Scatter(x=target_datetime_utc, y=expected_generation_kw, mode='lines+markers'))
fig.update_layout(
    title='Expected Generation (kW) Over Time',
    xaxis_title='Target Datetime UTC',
    yaxis_title='Expected Generation (kW)',
    xaxis=dict(tickangle=45),
    template='plotly_white'
)
fig.show()


## Example POST live PV data to the system to be used for forecasting

In [ ]:
import datetime
site_uuid = 'INSERT_SITE_UUID'
headers = {
    "Authorization": "Bearer " + access_token,
    "Content-Type": "application/json"
}

live_pv_data = {
    "site_uuid": site_uuid,
    "pv_actual_values": [
        {
            "datetime_utc": datetime.datetime.utcnow().isoformat() + "Z",
            "actual_generation_kw": 0.05,
        }
    ]
}

# Correcting the URL string formatting for the POST request
response = requests.post(base_url + f"/sites/{site_uuid}/pv_actual", headers=headers, json=live_pv_data)
if response.status_code == 200:
    print("Data posted successfully:", response.json())
else:
    print("Failed to post data. Status code:", response.status_code, "Response:", response.json())